# Set Up

## Installs

In [ ]:
pip install numpy accelerate diffusers ftfy bitsandbytes safetensors wandb datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
%pip install -qq git+https://github.com/huggingface/diffusers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!wget -q https://raw.githubusercontent.com/huggingface/diffusers/main/examples/textual_inversion/textual_inversion.py

In [ ]:
!wget -q https://raw.githubusercontent.com/huggingface/diffusers/main/examples/textual_inversion/textual_inversion_sdxl.py

In [ ]:
# Clone the repo into a temporary directory to gain access to the necessary utils
!git clone https://github.com/xyzLeander/businessheadshotsthesis.git temp

# Move the contents of the temporary directory to current working directory
!mv temp/* .

# Remove the temporary directory
!rm -r temp

## Imports

In [ ]:
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline, DDIMScheduler, DDPMScheduler, DPMSolverMultistepScheduler
from diffusers.utils import make_image_grid
from string import whitespace
from google.colab import userdata, drive, files
from collections import OrderedDict
from safetensors.torch import load_file
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
from datasets import load_dataset

import torch
import os
import shutil
import tensorflow as tf
import datetime
import itertools
import matplotlib.pyplot as plt
import numpy as np
import datasets
import json
import requests
import wandb
import sys
import prep_utils

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

## Environment config

In [ ]:
!accelerate config default # setup accelerate config

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
!wandb login # login weights & biases

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Colab Secrets

In [ ]:
os.environ['HF_TOKEN']= userdata.get('Huggingface') # acces huggingface token

## Prep

In [ ]:
drive.mount('/content/drive') # access google drive

Mounted at /content/drive


### Setup Single Training Run

In [ ]:
# key variables
base_dir = '/content/drive/MyDrive/training_images' # path to all training images in google drive
train_directory = '/content/train_images' # path to move the training images to
results_dir = '/content/drive/My Drive/results' # path to result models/embeddings
subfolders = ['textinv', 'dbooth', 'dlora', 'lora'] # subfolders for creating the folder structure

# adjust the parameters below depending on the training process
training_method = 'textinv' # training method: "textinv"
training_model = 'SDXL' # model version: "SDv1.5" or "SDXL"
training_subject = 'leander' # name of subject to train (or directory name referencing the correct subject)

prep_utils.create_folder_structure(results_dir, subfolders) # prepare directory structure in google drive
prep_utils.setup_training_environment(train_directory, results_dir, training_model, training_method, training_subject) # setup for traning

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Root directory already exists: /content/drive/My Drive/results
Subfolder already exists: /content/drive/My Drive/results/textinv
Sub-subfolder already exists: /content/drive/My Drive/results/textinv/SDXL
Sub-subfolder already exists: /content/drive/My Drive/results/textinv/SDv1.5
Sub-subfolder already exists: /content/drive/My Drive/results/textinv/flux
Subfolder already exists: /content/drive/My Drive/results/dbooth
Sub-subfolder already exists: /content/drive/My Drive/results/dbooth/SDXL
Sub-subfolder already exists: /content/drive/My Drive/results/dbooth/SDv1.5
Sub-subfolder already exists: /content/drive/My Drive/results/dbooth/flux
Subfolder already exists: /content/drive/My Drive/results/dlora
Sub-subfolder already exists: /content/drive/My Drive/results/dlora/SDXL
Sub-subfolder already exists: /content/drive/My Drive/results/dlora/SDv1.5
Sub-subfolder 

In [ ]:
images = prep_utils.load_images(base_dir, training_subject) # load subject images from drive
prep_utils.move_images(training_subject, images, train_directory) # move images to environment

### Setup Training Loop

In [ ]:
# key variables
base_dir = '/content/drive/MyDrive/training_images' # path to all training images in google drive
train_directory = '/content/train_images' # path to move the training images to
results_dir = '/content/drive/My Drive/results' # path to result models/embeddings
subfolders = ['textinv', 'dbooth', 'dlora', 'lora'] # subfolders for creating the folder structure

subject_list = ['hannah','timh', 'celine', 'marco', 'jannik', 'christoph', 'patrick','timk', 'leander', 'nils'] # list of directory names that reference all subjects

# adjust the parameters below depending on the training process
training_method = 'textinv' # training method: "textinv"
training_model = 'SDXL' # model version: "SDv1.5" or "SDXL"

prep_utils.create_folder_structure(results_dir, subfolders) # prepare directory structure in google drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Root directory already exists: /content/drive/My Drive/results
Subfolder already exists: /content/drive/My Drive/results/textinv
Sub-subfolder already exists: /content/drive/My Drive/results/textinv/SDXL
Sub-subfolder already exists: /content/drive/My Drive/results/textinv/SDv1.5
Sub-subfolder already exists: /content/drive/My Drive/results/textinv/flux
Subfolder already exists: /content/drive/My Drive/results/dbooth
Sub-subfolder already exists: /content/drive/My Drive/results/dbooth/SDXL
Sub-subfolder already exists: /content/drive/My Drive/results/dbooth/SDv1.5
Sub-subfolder already exists: /content/drive/My Drive/results/dbooth/flux
Subfolder already exists: /content/drive/My Drive/results/dlora
Sub-subfolder already exists: /content/drive/My Drive/results/dlora/SDXL
Sub-subfolder already exists: /content/drive/My Drive/results/dlora/SDv1.5
Sub-subfolder 

# Trainings

## Training textual inversion embed (SD 1.5)

In [ ]:
# check environment variables before running training
!echo $TRAIN_DIR
!echo $OUTPUT_DIR
!echo $MODEL_NAME
!echo $VALID_PROMPT
!echo $RESOLUTION
!echo $PLACEHOLDER_TOKEN
!echo $INITIALIZER_TOKEN

/content/train_images
/content/drive/My Drive/results/textinv/SDXL/leander
stabilityai/stable-diffusion-xl-base-1.0
business headshot of a <sks> man
1024
<sks>
man


In [ ]:
# Check if the MODEL_NAME is set correctly
if os.environ.get('MODEL_NAME') == 'benjamin-paine/stable-diffusion-v1-5':
    # Define the parameters for the training script
    train_params = {
        "pretrained_model_name_or_path": os.environ.get('MODEL_NAME'),
        "train_data_dir": os.environ.get('TRAIN_DIR'),
        "learnable_property": "object",
        "placeholder_token": os.environ.get('PLACEHOLDER_TOKEN'),
        "initializer_token": os.environ.get('INITIALIZER_TOKEN'),
        "resolution": os.environ.get('RESOLUTION'),
        "train_batch_size": 4,
        "num_vectors": 8,
        "gradient_accumulation_steps": 1,
        "max_train_steps": 2000,
        "learning_rate": 1e-04,
        # "scale_lr": True,
        "lr_scheduler": "constant",
        "lr_warmup_steps": 0,
        "validation_prompt": os.environ.get('VALID_PROMPT'),
        "validation_steps":100,
        "output_dir": os.environ.get('OUTPUT_DIR'),
        "mixed_precision": "fp16",
        "checkpointing_steps": 100,
        "report_to": "wandb",
        "seed": 42,
    }

    # Save the parameters to a text file
    params_file = os.path.join(os.environ.get('OUTPUT_DIR'), "training_params.txt")
    with open(params_file, "w") as f:
        for key, value in train_params.items():
            f.write(f"{key}: {value}\n")

    # Construct the command to run the training script
    !accelerate launch textual_inversion.py \
        --pretrained_model_name_or_path="{train_params['pretrained_model_name_or_path']}"  \
        --output_dir="{train_params['output_dir']}" \
        --train_data_dir="{train_params['train_data_dir']}" \
        --resolution={train_params['resolution']} \
        --learnable_property="{train_params['learnable_property']}" \
        --initializer_token="{train_params['initializer_token']}" \
        --placeholder_token="{train_params['placeholder_token']}" \
        --train_batch_size="{train_params['train_batch_size']}" \
        --gradient_accumulation_steps={train_params['gradient_accumulation_steps']} \
        --num_vectors={train_params['num_vectors']} \
        --resume_from_checkpoint="latest" \
        --max_train_steps={train_params['max_train_steps']} \
        --checkpointing_steps={train_params['checkpointing_steps']} \
        --mixed_precision="{train_params['mixed_precision']}" \
        --learning_rate={train_params['learning_rate']} --lr_scheduler="{train_params['lr_scheduler']}" --lr_warmup_steps={train_params['lr_warmup_steps']} \
        --validation_steps={train_params['validation_steps']} --validation_prompt="{train_params['validation_prompt']}" --report_to="{train_params['report_to']}" \
        --seed={train_params['seed']}
else:
    print("Cannot run training script. Environment variable MODEL_NAME is not set correctly. Set MODEL_NAME correctly or use appropriate training script.")

Cannot run training script. Environment variable MODEL_NAME is not set correctly. Set MODEL_NAME correctly or use appropriate training script.


## Training Loop (SD 1.5)

In [ ]:
for subject in subject_list:
  prep_utils.empty_directory(train_directory) # empties train_directory to prepare for training of the next subject

  training_subject = subject

  prep_utils.setup_training_environment(train_directory, results_dir, training_model, training_method, training_subject)
  images = prep_utils.load_images(base_dir, training_subject)
  prep_utils.move_images(training_subject, images, train_directory)

  # Check if the MODEL_NAME is set correctly
  if os.environ.get('MODEL_NAME') == 'benjamin-paine/stable-diffusion-v1-5':
      # Define the parameters for the training script
      train_params = {
          "pretrained_model_name_or_path": os.environ.get('MODEL_NAME'),
          "train_data_dir": os.environ.get('TRAIN_DIR'),
          "learnable_property": "object",
          "placeholder_token": os.environ.get('PLACEHOLDER_TOKEN'),
          "initializer_token": os.environ.get('INITIALIZER_TOKEN'),
          "resolution": os.environ.get('RESOLUTION'),
          "train_batch_size": 4,
          "num_vectors": 8,
          "gradient_accumulation_steps": 1,
          "max_train_steps": 2000,
          "learning_rate": 1e-04,
          # "scale_lr": True,
          "lr_scheduler": "constant",
          "lr_warmup_steps": 0,
          "validation_prompt": os.environ.get('VALID_PROMPT'),
          "validation_steps":100,
          "output_dir": os.environ.get('OUTPUT_DIR'),
          "mixed_precision": "fp16",
          "checkpointing_steps": 100,
          "report_to": "wandb",
          "seed": 42,
      }

      # Save the parameters to a text file
      params_file = os.path.join(os.environ.get('OUTPUT_DIR'), "training_params.txt")
      with open(params_file, "w") as f:
          for key, value in train_params.items():
              f.write(f"{key}: {value}\n")

      # Construct the command to run the training script
      !accelerate launch textual_inversion.py \
          --pretrained_model_name_or_path="{train_params['pretrained_model_name_or_path']}"  \
          --output_dir="{train_params['output_dir']}" \
          --train_data_dir="{train_params['train_data_dir']}" \
          --resolution={train_params['resolution']} \
          --learnable_property="{train_params['learnable_property']}" \
          --initializer_token="{train_params['initializer_token']}" \
          --placeholder_token="{train_params['placeholder_token']}" \
          --train_batch_size="{train_params['train_batch_size']}" \
          --gradient_accumulation_steps={train_params['gradient_accumulation_steps']} \
          --num_vectors={train_params['num_vectors']} \
          --resume_from_checkpoint="latest" \
          --max_train_steps={train_params['max_train_steps']} \
          --checkpointing_steps={train_params['checkpointing_steps']} \
          --mixed_precision="{train_params['mixed_precision']}" \
          --learning_rate={train_params['learning_rate']} --lr_scheduler="{train_params['lr_scheduler']}" --lr_warmup_steps={train_params['lr_warmup_steps']} \
          --validation_steps={train_params['validation_steps']} --validation_prompt="{train_params['validation_prompt']}" --report_to="{train_params['report_to']}" \
          --seed={train_params['seed']}
  else:
      print("Cannot run training script. Environment variable MODEL_NAME is not set correctly. Set MODEL_NAME correctly or use appropriate training script.")

  prep_utils.empty_directory(train_directory) # empties train_directory to prepare for training of the next subject


Directory does not exist: /content/train_images
Saved /content/train_images/timk_1.jpg
Saved /content/train_images/timk_2.jpg
Saved /content/train_images/timk_3.jpg
Saved /content/train_images/timk_4.jpg
Saved /content/train_images/timk_5.jpg
Saved /content/train_images/timk_6.jpg
Saved /content/train_images/timk_7.jpg
Saved /content/train_images/timk_8.jpg
Saved /content/train_images/timk_9.jpg
Saved /content/train_images/timk_10.jpg
2024-11-06 11:01:47.545880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 11:01:47.567149: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 11:01:47.573493: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting t

## Training Text Inversion Embed (SDXL)

In [ ]:
# Check if the MODEL_NAME is set correctly
if os.environ.get('MODEL_NAME') == 'stabilityai/stable-diffusion-xl-base-1.0':
    # Define the parameters for the training script
    train_params = {
        "pretrained_model_name_or_path": os.environ.get('MODEL_NAME'),
        "train_data_dir": os.environ.get('TRAIN_DIR'),
        "learnable_property": "object",
        "placeholder_token": os.environ.get('PLACEHOLDER_TOKEN'),
        "initializer_token": os.environ.get('INITIALIZER_TOKEN'),
        "resolution": os.environ.get('RESOLUTION'),
        "train_batch_size": 4,
        "num_vectors": 2,
        "gradient_accumulation_steps": 1,
        "max_train_steps": 5000,
        "learning_rate": 5e-03,
        "lr_scheduler": "cosine",
        "lr_warmup_steps": 0,
        "validation_prompt": os.environ.get('VALID_PROMPT'),
        "validation_steps":100,
        "output_dir": os.environ.get('OUTPUT_DIR'),
        "mixed_precision": "bf16",
        "checkpointing_steps": 500,
        "report_to": "wandb",
        "seed": 42,
    }

    Save the parameters to a text file
    params_file = os.path.join(os.environ.get('OUTPUT_DIR'), "training_params.txt")
    with open(params_file, "w") as f:
        for key, value in train_params.items():
            f.write(f"{key}: {value}\n")

    # Construct the command to run the training script
    !accelerate launch textual_inversion_sdxl.py \
        --pretrained_model_name_or_path="{train_params['pretrained_model_name_or_path']}"  \
        --output_dir="{train_params['output_dir']}" \
        --train_data_dir="{train_params['train_data_dir']}" \
        --resolution={train_params['resolution']} \
        --learnable_property="{train_params['learnable_property']}" \
        --initializer_token="{train_params['initializer_token']}" \
        --placeholder_token="{train_params['placeholder_token']}" \
        --train_batch_size="{train_params['train_batch_size']}" \
        --gradient_accumulation_steps={train_params['gradient_accumulation_steps']} \
        --num_vectors={train_params['num_vectors']} \
        --resume_from_checkpoint="latest" \
        --max_train_steps={train_params['max_train_steps']} \
        --checkpointing_steps={train_params['checkpointing_steps']} \
        --mixed_precision="{train_params['mixed_precision']}" \
        --learning_rate={train_params['learning_rate']} --lr_scheduler="{train_params['lr_scheduler']}" --lr_warmup_steps={train_params['lr_warmup_steps']} \
        --validation_steps={train_params['validation_steps']} --validation_prompt="{train_params['validation_prompt']}" --report_to="{train_params['report_to']}" \
        --seed={train_params['seed']}
else:
    print("Cannot run training script. Environment variable MODEL_NAME is not set correctly. Set MODEL_NAME correctly or use appropriate training script.")

2024-10-31 22:16:04.485693: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-31 22:16:04.506770: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-31 22:16:04.513214: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-31 22:16:05.836010: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
10/31/2024 22:16:07 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: bf16

{'clip_sample_range', 'rescale_betas_zero_snr', 'thresholding', 'variance_

## Training Loop Textual Inversion (SDXL)

In [ ]:
for subject in subject_list:
  prep_utils.empty_directory(train_directory) # empties train_directory to prepare for training the next subject

  training_subject = subject

  prep_utils.setup_training_environment(train_directory, results_dir, training_model, training_method, training_subject)
  images = prep_utils.load_images(base_dir, training_subject)
  prep_utils.move_images(training_subject, images, train_directory)

  if os.environ.get('MODEL_NAME') == 'stabilityai/stable-diffusion-xl-base-1.0':
  # Check if the MODEL_NAME is set correctly
    train_params = {
            "pretrained_model_name_or_path": os.environ.get('MODEL_NAME'),
            "train_data_dir": os.environ.get('TRAIN_DIR'),
            "learnable_property": "object",
            "placeholder_token": os.environ.get('PLACEHOLDER_TOKEN'),
            "initializer_token": os.environ.get('INITIALIZER_TOKEN'),
            "resolution": os.environ.get('RESOLUTION'),
            "train_batch_size": 4,
            "num_vectors": 2,
            "gradient_accumulation_steps": 1,
            "max_train_steps": 5000,
            "learning_rate": 5e-03,
            "lr_scheduler": "cosine",
            "lr_warmup_steps": 0,
            "validation_prompt": os.environ.get('VALID_PROMPT'),
            "validation_steps":100,
            "output_dir": os.environ.get('OUTPUT_DIR'),
            "mixed_precision": "bf16",
            "checkpointing_steps": 500,
            "report_to": "wandb",
            "seed": 42,
      }

      # Save the parameters to a text file
    params_file = os.path.join(os.environ.get('OUTPUT_DIR'), "training_params.txt")
    with open(params_file, "w") as f:
        for key, value in train_params.items():
            f.write(f"{key}: {value}\n")

    # Construct the command to run the training script
    !accelerate launch textual_inversion_sdxl.py \
        --pretrained_model_name_or_path="{train_params['pretrained_model_name_or_path']}"  \
        --output_dir="{train_params['output_dir']}" \
        --train_data_dir="{train_params['train_data_dir']}" \
        --resolution={train_params['resolution']} \
        --learnable_property="{train_params['learnable_property']}" \
        --initializer_token="{train_params['initializer_token']}" \
        --placeholder_token="{train_params['placeholder_token']}" \
        --train_batch_size="{train_params['train_batch_size']}" \
        --gradient_accumulation_steps={train_params['gradient_accumulation_steps']} \
        --num_vectors={train_params['num_vectors']} \
        --resume_from_checkpoint="latest" \
        --max_train_steps={train_params['max_train_steps']} \
        --checkpointing_steps={train_params['checkpointing_steps']} \
        --mixed_precision="{train_params['mixed_precision']}" \
        --learning_rate={train_params['learning_rate']} --lr_scheduler="{train_params['lr_scheduler']}" --lr_warmup_steps={train_params['lr_warmup_steps']} \
        --validation_steps={train_params['validation_steps']} --validation_prompt="{train_params['validation_prompt']}" --report_to="{train_params['report_to']}" \
        --seed={train_params['seed']}
  else:
    print("Cannot run training script. Environment variable MODEL_NAME is not set correctly. Set MODEL_NAME correctly or use appropriate training script.")

  prep_utils.empty_directory(train_directory) # empties train_directory to prepare for training of the next subject

Directory does not exist: /content/train_images
Saved /content/train_images/patrick_1.jpg
Saved /content/train_images/patrick_2.jpg
Saved /content/train_images/patrick_3.jpg
Saved /content/train_images/patrick_4.jpg
Saved /content/train_images/patrick_5.jpg
Saved /content/train_images/patrick_6.jpg
Saved /content/train_images/patrick_7.jpg
Saved /content/train_images/patrick_8.jpg
Saved /content/train_images/patrick_9.jpg
Saved /content/train_images/patrick_10.jpg
2024-11-07 11:39:21.108113: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-07 11:39:21.128940: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-07 11:39:21.135224: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to registe